In [9]:
%load_ext autoreload
%autoreload 2
#Assessments with confirmed active results
#107617, 107614

# 17 mins for current operation with all ids
# 2 mins for 100 ids. 

#To DO
#Implement logging to the whole operations

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate, assessment_id_list
from airflow import DAG
from airflow.operators.python import PythonOperator
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')

access_token, expires_in = get_access_token()

test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group') #Seperate table
test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')

test_results_view = create_test_results_view(test_results_standard, access_token, '23-24') #add in grade level col, string matching

# #Write dfs to SFTP



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The length of the ID_list is 167
The length of the ID_list is 167
The length of the ID_list is 167
